# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

misschien handige code om te gebruiken:

%pwd (de data moet in je workdirectery staan)


In [2]:
import numpy as np
import pandas as pd

In [12]:
dfProjectTasks = pd.read_excel('MODPR dataset.xlsx', sheet_name ='Project tasks')

In [14]:
dfProjectTasks.head()

,Task,ReqSkill 1,ReqSkill 2,ReqSkill 3,ReqSkill 4,ReqSkill 5,ReqSkill 6,ReqSkill 7,ReqSkill 8,ReqSkill 9,...,ReqRoom **,ReqRoom ***,ReqRoom ****,ReqRoom 4,ReqRoom 5,MinReqCrew,MaxReqCrew,Frequency,Duration,Comments
0,S1C1T1A,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,...,x,x,x,NaN,NaN,2.0,NaN,4.0,2.0,NaN
1,S1C1T1B,NaN,NaN,NaN,x,NaN,NaN,x,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,2.0,NaN
2,S1C1T2,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,...,x,x,x,NaN,NaN,2.0,NaN,8.0,2.0,NaN
3,S1C1T3,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,x,x,x,NaN,NaN,2.0,NaN,8.0,2.0,NaN
4,S1C1T4,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,x,x,x,NaN,NaN,3.0,NaN,8.0,2.0,NaN


In [6]:
dfAdditionalTasks = pd.read_excel('MODPR dataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPR dataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPR dataset.xlsx', sheet_name ='Rooms')

'C:\\Users\\Robbert\\Github\\Modelleren'